In [17]:
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import time

def gerar_link_pasta_dodf(data):
    """
    Gera o link da pasta do DODF para uma data específica
    """
    data_obj = datetime.strptime(data, '%d/%m/%Y')
    
    meses = {
        1: 'Janeiro',
        2: 'Fevereiro',
        3: 'Marco',
        4: 'Abril',
        5: 'Maio',
        6: 'Junho',
        7: 'Julho',
        8: 'Agosto',
        9: 'Setembro',
        10: 'Outubro',
        11: 'Novembro',
        12: 'Dezembro'
    }
    
    ano = data_obj.strftime('%Y')
    mes_numero = data_obj.month
    mes_nome = meses[mes_numero]
    data_formatada = f"{data_obj.strftime('%d')} {data_obj.strftime('%m')} {data_obj.strftime('%Y')}"
    
    return f"https://dodf.df.gov.br/dodf/jornal/pastas?pasta={ano}/{mes_numero:02d}_{mes_nome}/{data_formatada}".replace(" ", "%20")

def obter_edicoes_do_dia(data):
    """
    Obtém as edições disponíveis para uma data específica usando Selenium
    """
    url = gerar_link_pasta_dodf(data)
    
    # Configurar opções do Chrome
    chrome_options = Options()
    chrome_options.add_argument('--headless')  # Executa em modo headless (sem interface gráfica)
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    
    try:
        # Iniciar o navegador
        driver = webdriver.Chrome(options=chrome_options)
        print(f"Acessando URL: {url}")
        
        # Acessar a URL
        driver.get(url)
        
        # Esperar que os elementos sejam carregados (máximo 20 segundos)
        wait = WebDriverWait(driver, 20)
        elementos = wait.until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "link-materia"))
        )
        
        # Dar um tempo extra para garantir que tudo carregou
        time.sleep(2)
        
        # Coletar informações dos PDFs
        pdfs = []
        elementos = driver.find_elements(By.CLASS_NAME, "link-materia")
        
        for elemento in elementos:
            try:
                nome = elemento.find_element(By.TAG_NAME, "p").text.strip()
                link = elemento.get_attribute("href")
                pdfs.append({
                    'nome': nome,
                    'link': link
                })
            except Exception as e:
                print(f"Erro ao processar elemento: {e}")
        
        return pdfs
        
    except Exception as e:
        print(f"Erro ao acessar a pasta: {e}")
        return None
        
    finally:
        try:
            driver.quit()
        except:
            pass



In [18]:
# Exemplo de uso
def main():
    data = "02/01/2024"
    print(f"Buscando edições para {data}...")
    
    edicoes = obter_edicoes_do_dia(data)
    
    if edicoes:
        print(f"\nEdições disponíveis para {data}:")
        for edicao in edicoes:
            print(f"\nNome: {edicao['nome']}")
            print(f"Link: {edicao['link']}")
    else:
        print(f"\nNão foi possível obter as edições para {data}")

if __name__ == "__main__":
    main()

Buscando edições para 02/01/2024...


Cache folder (/Users/henrique/.cache/selenium) cannot be created: Permission denied (os error 13)
Cache folder (/Users/henrique/.cache/selenium) cannot be created: Permission denied (os error 13)
Cache folder (/Users/henrique/.cache/selenium) cannot be created: Permission denied (os error 13)
Cache folder (/Users/henrique/.cache/selenium) cannot be created: Permission denied (os error 13)


Acessando URL: https://dodf.df.gov.br/dodf/jornal/pastas?pasta=2024/01_Janeiro/02%2001%202024

Edições disponíveis para 02/01/2024:

Nome: DODF 001 02-01-2024 INTEGRA.pdf
Link: https://dodf.df.gov.br/dodf/jornal/visualizar-pdf?pasta=2024|01_Janeiro|DODF%20001%2002-01-2024|&arquivo=DODF%20001%2002-01-2024%20INTEGRA.pdf

Nome: DODF 001 02-01-2024 EDICAO EXTRA A.pdf
Link: https://dodf.df.gov.br/dodf/jornal/visualizar-pdf?pasta=2024|01_Janeiro|DODF%20001%2002-01-2024%20EDICAO%20EXTRA%20A|&arquivo=DODF%20001%2002-01-2024%20EDICAO%20EXTRA%20A.pdf


/var/folders/fj/821tv6qn54xbnrgw2tjk7jf00000gn/T/ipykernel_59708/2578908412.py:73: DeprecationWarning: using WebElement.get_attribute() has been deprecated. Please use get_dom_attribute() instead.
  link = elemento.get_attribute("href")
